In [ ]:
# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
#%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import dataiku

# Récupère le dataset
df_rfm_client_description_features = dataiku.Dataset("df_rfm_client_description_features")
df = df_rfm_client_description_features.get_dataframe()

# One-hot encoding des segments RFM
df = pd.get_dummies(df, columns=["rfm_segment"], prefix="segment")

# Sélection des colonnes pour le clustering
X = df[["frequency", "monetary", "recency_days"] + [col for col in df.columns if col.startswith('segment_')]]

# Standardisation des données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Réduction de dimensionnalité avec PCA
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)

# Ajoute les composantes PCA au dataframe
df['pca1'] = X_pca[:, 0]
df['pca2'] = X_pca[:, 1]
df['pca3'] = X_pca[:, 2]

# Applique K-Means sur les données standardisées
kmeans = KMeans(n_clusters=5, random_state=42)
df['cluster'] = kmeans.fit_predict(X_scaled)

# Visualisation 3D des clusters
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(df['pca1'], df['pca2'], df['pca3'], c=df['cluster'], cmap='viridis', s=50)
ax.set_xlabel('Composante PCA 1')
ax.set_ylabel('Composante PCA 2')
ax.set_zlabel('Composante PCA 3')
ax.set_title('Visualisation 3D des Clusters (PCA)')
plt.colorbar(scatter, ax=ax, label='Cluster')
plt.tight_layout()

# Affiche l'image directement dans le notebook
#plt.show()

# Sauvegarde le dataframe avec les clusters
df_rfm_with_clusters = dataiku.Dataset("df_rfm_kmeans_pca")
df_rfm_with_clusters.write_with_schema(df)

"""

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import dataiku
import plotly.express as px
import plotly.io as pio
from plotly.offline import init_notebook_mode

# Initialise Plotly pour le notebook
init_notebook_mode(connected=True)
pio.renderers.default = 'notebook'  # Force l'affichage dans le notebook

# Récupère le dataset
df_rfm_client_description_features = dataiku.Dataset("df_rfm_client_description_features")
df = df_rfm_client_description_features.get_dataframe()

# One-hot encoding des segments RFM
df = pd.get_dummies(df, columns=["rfm_segment"], prefix="segment")

# Sélection des colonnes pour le clustering
X = df[["frequency", "monetary", "recency_days"] + [col for col in df.columns if col.startswith('segment_')]]

# Standardisation des données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Réduction de dimensionnalité avec PCA
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)

# Ajoute les composantes PCA au dataframe
df['pca1'] = X_pca[:, 0]
df['pca2'] = X_pca[:, 1]
df['pca3'] = X_pca[:, 2]

# Applique K-Means sur les données standardisées
kmeans = KMeans(n_clusters=5, random_state=42)
df['cluster'] = kmeans.fit_predict(X_scaled)

# Visualisation 3D des clusters avec Plotly
fig = px.scatter_3d(
    df,
    x='pca1',
    y='pca2',
    z='pca3',
    color='cluster',
    title='Visualisation 3D des Clusters (PCA)',
    labels={'pca1': 'Composante PCA 1', 'pca2': 'Composante PCA 2', 'pca3': 'Composante PCA 3'},
    hover_data=['frequency', 'monetary', 'recency_days']  # Affiche les valeurs au survol
)

# Affiche le graphique interactif
pio.show(fig)  # Utilise plotly.io.show au lieu de fig.show()

# Sauvegarde le dataframe avec les clusters
df_rfm_with_clusters = dataiku.Dataset("df_rfm_kmeans_pca")
df_rfm_with_clusters.write_with_schema(df)
"""